In [14]:
import os
import random
import shutil
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import os
from scipy.stats import linregress

# Moving random file to test data dir

Successfully moved 300 files to FINALDATA/Test_data/ with their folder names appended.


In [194]:
import os
import random
import shutil

def select_and_copy_random_files(directories, destination_dir, total_files=300):
    # Get the list of all files from each directory
    all_files = []
    for directory in directories:
        files = os.listdir(directory)
        all_files.extend(files)

    # Ensure there are enough files to select from
    if len(all_files) < total_files:
        raise ValueError(f"Not enough files in total. Only {len(all_files)} files available.")

    # Randomly select the required number of files
    selected_files = random.sample(all_files, total_files)

    # Shuffle the selected files (optional, to add randomness)
    random.shuffle(selected_files)

    # Ensure the destination directory exists
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Copy the selected files to the destination directory
    for file in selected_files:
        source_file = None
        folder_name = None
        
        # Loop through directories to find the source of the file
        for directory in directories:
            if file in os.listdir(directory):
                source_file = os.path.join(directory, file)
                folder_name = os.path.basename(directory)  # Folder name as part of the new filename
                break
        
        # Create a custom renaming scheme. For example: 'folder_name_original_filename'
        new_file_name = f"{folder_name}_{file}"
        destination_file = os.path.join(destination_dir, new_file_name)

        # Copy the file (instead of moving it)
        shutil.copy(source_file, destination_file)

    print(f"Successfully copied {total_files} files to {destination_dir} with the renaming scheme.")

# List of directories to select files from
directories = [
    "FINALDATA/Split_Data/Evolved+disk/EDNS",
    "FINALDATA/Split_Data/Evolved+disk/EDS",
    "FINALDATA/Split_Data/Evolved/ENS",
    "FINALDATA/Split_Data/Evolved/ES",  # Add your 4th directory
    "FINALDATA/Split_Data/WT/WTNS",  # Add your 5th directory
    "FINALDATA/Split_Data/WT/WTS",  # Add your 6th directory
]

destination_dir = "FINALDATA/Test_data3/"

select_and_copy_random_files(directories, destination_dir, total_files=500)


Successfully copied 500 files to FINALDATA/Test_data3/ with the renaming scheme.


# Definitions

In [6]:
def mean_squared_displacement_optimised(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = np.array(data['time'])
    
    msd_by_tau = {}

    for tau in range(1, len(time)):
        
        dx = x_smooth[tau:] - x_smooth[:-tau]
        dy = y_smooth[tau:] - y_smooth[:-tau]
        dz = z_smooth[tau:] - z_smooth[:-tau]
        squared_displacements = dx**2 + dy**2 + dz**2
        msd_by_tau[round(time[tau] - time[0], 3)] = np.mean(squared_displacements)

    return msd_by_tau


def find_gradient_intercept(file_path):
    msd_by_tau = mean_squared_displacement_optimised(file_path)
    
    # Check if msd_by_tau has enough data to perform regression
    if len(msd_by_tau) < 2:
        return np.nan, np.nan  # Not enough data to perform linear regression
    
    tau = np.array(list(msd_by_tau.keys()))
    msd_arr = np.array(list(msd_by_tau.values()))
    
    log_tau = np.log(tau)
    log_msd = np.log(msd_arr)
    
    # Perform linear regression if the data is valid
    try:
        slope, intercept, _, _, _ = linregress(log_tau, log_msd)
        return slope, intercept
    except ValueError:
        return np.nan, np.nan

In [7]:
def calc_curvature_arr(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    if len(data) <= 2:
        return np.array([])
    v_arr = np.zeros(len(data)) #init velocity array 
    t_arr = np.zeros(len(data))  #init time array    
    T_arr = np.zeros((len(data)-2, 4))  # 2D array with time + 3 components t, x,y,z

    for i in range(1, len(data)-1):

    
        x_smooth = np.array(data['x_smooth'])  #Extracting data
        y_smooth = np.array(data['y_smooth'])
        z_smooth = np.array(data['z_smooth'])
        time = np.array(data['time'])
    
        t_next = time[i+1]       #Define points for central difference
        x_next = x_smooth[i+1]
        y_next = y_smooth[i+1]
        z_next = z_smooth[i+1]
    
        t_prev = time[i-1]
        x_prev = x_smooth[i-1]
        y_prev = y_smooth[i-1]
        z_prev = z_smooth[i-1]
        dir_vector = np.array([x_next - x_prev, y_next - y_prev, z_next - z_prev])
        T_vector = dir_vector / np.linalg.norm(dir_vector)
        T_arr[i-1] = np.array([time[i], *T_vector])  
    
        r_diff = np.sqrt((x_next-x_prev)**2+(y_next-y_prev)**2+(z_next-z_prev)**2)
        dt = t_next - t_prev
        v = r_diff/ (dt)

        v_arr[i] = v
    time_for_T = time[1:-1] # Time associated with the T vectors, lost first and last point 
        
    v_arr = v_arr[2:-2]    #velocity for final calculation. Los first and last two points
        

    dT_arr = np.zeros((len(T_arr)-2, 4))  #init dT array, loses frist and last point from T_arr
    
    for i in range(1, len(T_arr)-1):
        dT = (T_arr[i+1, 1:] - T_arr[i-1, 1:]) / (T_arr[i+1, 0] - T_arr[i-1, 0])   #central diff for dT/dt 
        dT_arr[i-1] = np.array([T_arr[i, 0], *dT])#Put associated t x, y ,z in array
        
    curvature_arr = np.zeros(len(dT_arr))    #init curve array 
    
    for i in range(len(v_arr)):
        
        dT_segment = dT_arr[i, 1:]    #extract only x y z 
        dT_magnitude = np.linalg.norm(dT_segment)  #mag of x y z vector 
        v_mag = v_arr[i] #veloctity associated with this point 
        
        curvature = dT_magnitude / v_mag  #calculate curvature
        curvature_arr[i] = curvature   #put in array 
    times_for_curvature = time[2:-2]   #Time associated with the curvature array.
    
    return curvature_arr , times_for_curvature  #Return curvatre and associated time array for plotting.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def calc_acc(file_path):
    
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    if len(data) <= 2:
        return np.array([])
    v_arr = np.zeros(len(data))
    t_arr = np.zeros(len(data)-1)
    a_arr = np.zeros(len(data)-2)
    for i in range(1,len(data)-1):
        data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
        x_smooth = np.array(data['x_smooth'])
        y_smooth = np.array(data['y_smooth'])
        z_smooth = np.array(data['z_smooth'])
        time = np.array(data['time'])
    
        t_next = time[i+1]
        x_next = x_smooth[i+1]
        y_next = y_smooth[i+1]
        z_next = z_smooth[i+1]
    
        t_prev = time[i-1]
        x_prev = x_smooth[i-1]
        y_prev = y_smooth[i-1]
        z_prev = z_smooth[i-1]
        r_diff = np.sqrt((x_next-x_prev)**2+(y_next-y_prev)**2+(z_next-z_prev)**2)
        dt = t_next - t_prev
  
  
    #velocity at the current point using central difference
        
    
        v_arr[i] = r_diff / dt
    
    t_arr = time[1:-1]
    v_arr = v_arr[1:-1]

        
    for i in range(1, len(v_arr) - 1):
        dv = (v_arr[i + 1] - v_arr[i - 1])
        dt = time[i + 1] - time[i - 1]
        a_arr[i] = dv / dt
    t_for_acc = time [2:-2] 
    a_arr = a_arr[1:-1]   
    return a_arr , t_for_acc
    

In [9]:
def calc_velo_arr(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = np.array(data['time'])
    if len(data) <= 2:
        return np.array([])
    v_arr = np.zeros(len(data)-2)
    
 
    

    for i in range(1,len(data)-1):
        t_next = time[i+1]
        x_next = x_smooth[i+1]
        y_next = y_smooth[i+1]
        z_next = z_smooth[i+1]
    
        t_prev = time[i-1]
        x_prev = x_smooth[i-1]
        y_prev = y_smooth[i-1]
        z_prev = z_smooth[i-1]
    
    # Calculate radial distance for next and previous points
 
    
    
        r_diff = np.sqrt((x_next-x_prev)**2+(y_next-y_prev)**2+(z_next-z_prev)**2)
        dt = t_next - t_prev
  
  
    #velocity at the current point using central difference
        v = r_diff/ (dt)

        v_arr[i-1] = v
        
    return v_arr
        

In [10]:

def calculate_path_length(file_path):
    # Read the data from the .txt file
    data = pd.read_csv(file_path, delim_whitespace=True, header=None,
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
    # Extract x_smooth, y_smooth, and z_smooth for path length calculation
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])

    # Calculate the path length as the sum of the distances between consecutive points
    path_length = np.sum(np.sqrt(np.diff(x_smooth)**2 + np.diff(y_smooth)**2 + np.diff(z_smooth)**2))
    
    return path_length

def calculate_straight_distance(file_path):
    # Read the data from the .txt file
    data = pd.read_csv(file_path, delim_whitespace=True, header=None,
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
    # Extract starting and ending coordinates
    start_point = data.iloc[0][['x_smooth', 'y_smooth', 'z_smooth']].to_numpy()
    end_point = data.iloc[-1][['x_smooth', 'y_smooth', 'z_smooth']].to_numpy()
    
    # Calculate the straight-line distance
    straight_distance = np.linalg.norm(end_point - start_point)
    
    return straight_distance

def calculate_tortuosity(file_path):
    path_length = calculate_path_length(file_path)
    straight_distance = calculate_straight_distance(file_path)
    
    # Avoid division by zero
    if straight_distance == 0:
        return np.inf  # Tortuosity is infinite if the straight distance is zero
    
    # Tortuosity calculation
    tortuosity = path_length / straight_distance
    return tortuosity

In [11]:
def reorientation_events_per_second(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None,
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = data['time']
    
    positions = np.stack((x_smooth, y_smooth, z_smooth), axis=1)

    # Calculate differences in positional vectors between consecutive points
    dir_vectors = np.diff(positions, axis=0)
    # Calculate magnitudes of vectors
    mags = np.linalg.norm(dir_vectors, axis=1)

    # Avoid division by zero
    non_zero_mags = mags > 0  
    unit_dir = np.zeros_like(dir_vectors)  
    unit_dir[non_zero_mags] = dir_vectors[non_zero_mags] / mags[non_zero_mags, np.newaxis]

    # Calculate dot products k steps ahead
    k = 5
    dot_products = [np.dot(unit_dir[i], unit_dir[i + k]) for i in range(len(unit_dir) - k) if i + k < len(unit_dir)]

    change_in_dir_threshold = 0.98
    consecutive_low_values = 3

    low_sequences = []
    current_sequence = []

    for dp in dot_products:
        if dp < change_in_dir_threshold:
            current_sequence.append(dp)
        else:
            if len(current_sequence) >= consecutive_low_values:
                low_sequences.append(current_sequence)
            current_sequence = []  # Reset for the next sequence

    if len(current_sequence) >= consecutive_low_values:
        low_sequences.append(current_sequence)

    # Calculate events per second
    events_per_sec = len(low_sequences) / (np.max(time) - np.min(time))
    return events_per_sec

# Creating DF

In [12]:
import os
import pandas as pd
import numpy as np

def extract_features_to_array(file_path, min_length=10):
    # Extract features from file
    msd_by_tau = mean_squared_displacement_optimised(file_path)
    velocity = calc_velo_arr(file_path)
    acceleration = calc_acc(file_path)
    reorientations_per_sec = reorientation_events_per_second(file_path)
    curvature = calc_curvature_arr(file_path)
    tortuosity = calculate_tortuosity(file_path)
    path_length = calculate_path_length(file_path)
    slope, intercept = find_gradient_intercept(file_path)
    
    # Check the length of the data (if track is too short, skip it)
    if len(msd_by_tau) < min_length or len(velocity) < min_length:
        return None  # Skip tracks that are too short
    
    # Ensure features arrays have values, or use np.nan if empty
    msd_arr = np.array(list(msd_by_tau.values())) if len(msd_by_tau) > 0 else np.nan
    log_velocity = np.log(velocity[velocity > 0]) if len(velocity) > 0 else np.nan
    
    # Return exactly 21 features (track_type, filename, and 19 features)
    features = [
        
        slope if slope else np.nan,
        intercept if intercept else np.nan,
        # Placeholder for gradient and intercept (if not calculated)
        np.mean(log_velocity) if len(log_velocity) > 0 else np.nan,
        np.std(log_velocity) if len(log_velocity) > 0 else np.nan,
        np.max(log_velocity) if len(log_velocity) > 0 else np.nan,
        np.min(log_velocity) if len(log_velocity) > 0 else np.nan,  # Log velocity features
        np.mean(acceleration) if len(acceleration) > 0 else np.nan, 
        np.std(acceleration) if len(acceleration) > 0 else np.nan,
        np.max(acceleration) if len(acceleration) > 0 else np.nan, 
        np.min(acceleration) if len(acceleration) > 0 else np.nan,  # Acceleration features
        reorientations_per_sec if reorientations_per_sec else np.nan,  # Reorientation events
        np.mean(curvature) if len(curvature) > 0 else np.nan,
        np.std(curvature) if len(curvature) > 0 else np.nan,
        np.max(curvature) if len(curvature) > 0 else np.nan,  # Curvature features
        tortuosity if tortuosity else np.nan, 
        path_length if path_length else np.nan  # Tortuosity and path length
    ]
    
    return features

def feature_df(directory_path, min_length=10):
    all_features = []  # List to store valid feature data
    
    # Iterate through files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        if os.path.isfile(file_path):  # Ensure it's a file
            features = extract_features_to_array(file_path, min_length)
            
            if features is not None:  # Only add valid tracks
                track_type = filename.split('_')[0]  # Assuming type is in the filename
                all_features.append([track_type, filename] + features)
    
    # Remove any None values (files that didn't meet length threshold)
    all_features = [feature for feature in all_features if feature is not None]
    
    # Convert list of valid features to a DataFrame
    df = pd.DataFrame(all_features, columns=[
        'track_type', 'filename',
        'gradient', 'intercept', 'mean_log_velocity', 'stddev_log_velocity',
        'max_log_velocity', 'min_log_velocity', 'mean_acceleration', 'stddev_acceleration',
        'max_acceleration', 'min_acceleration', 'reorientations_per_sec', 'mean_curvature',
        'stddev_curvature', 'max_curvature', 'tortuosity', 'path_length'
    ])
    
    return df





In [15]:
df = feature_df('FINALDATA/Test_data3/')

<ipython-input-7-adf42148a9bd>:28: RuntimeWarning: invalid value encountered in true_divide
  T_vector = dir_vector / np.linalg.norm(dir_vector)
<ipython-input-7-adf42148a9bd>:55: RuntimeWarning: invalid value encountered in double_scalars
  curvature = dT_magnitude / v_mag  #calculate curvature
<ipython-input-7-adf42148a9bd>:55: RuntimeWarning: divide by zero encountered in double_scalars
  curvature = dT_magnitude / v_mag  #calculate curvature


In [17]:
df

,track_type,filename,gradient,intercept,mean_log_velocity,stddev_log_velocity,max_log_velocity,min_log_velocity,mean_acceleration,stddev_acceleration,max_acceleration,min_acceleration,reorientations_per_sec,mean_curvature,stddev_curvature,max_curvature,tortuosity,path_length
0,ENS,ENS_track81208_0_processed.txt,1.701232,4.237139,2.133661,0.898218,3.536060,-0.360211,9.557535,248.384852,968.808509,-606.917541,6.976744,9.561206,24.157304,263.837759,1.776933,10.287340
1,EDNS,EDNS_track54912_0_processed.txt,1.606683,4.109765,2.447608,0.644055,3.514067,0.068679,11.401612,242.850418,762.773742,-848.551621,9.917355,6.813065,11.564500,141.513504,1.907978,8.311156
2,ES,ES_track132939_0_processed.txt,1.801404,6.427257,3.098041,0.744236,4.277293,1.206329,6.949521,246.013576,972.965287,-1097.691499,6.930693,6.025760,5.480151,31.300272,1.554178,28.870975
3,EDNS,EDNS_track34474_0_processed.txt,0.890607,0.350629,1.490617,1.195546,2.680981,-2.453408,11.947559,107.573897,363.710511,-410.032948,7.734807,20.707341,134.421856,2253.366614,5.884277,6.002668
4,EDNS,EDNS_track92165_0_processed.txt,0.754142,1.276057,1.964142,1.042575,3.739174,-0.719390,33.195348,225.044808,1122.456099,-707.501607,8.474576,15.119860,36.129708,402.859431,2.897660,7.020181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,ENS,ENS_track252767_0_processed.txt,1.675585,2.109512,1.427313,0.538436,2.083858,0.119277,10.156549,49.253875,176.379752,-159.076280,5.106383,13.662256,10.839919,81.865328,1.706384,5.552453
810,WTS,WTS_track21_0 (2)_processed.txt,1.879332,5.638844,2.885362,0.229033,3.586314,2.011337,2.677165,69.158460,378.385373,-700.869047,6.238532,2.261897,2.493457,8.175000,1.306811,150.303455
811,EDNS,EDNS_track129486_0_processed.txt,1.468388,5.684454,3.246148,0.702201,3.969870,0.790571,31.706242,225.128408,1357.149690,-790.622786,6.593407,14.201588,13.474938,59.022597,1.858490,28.252718
812,ES,ES_track22172_0_processed.txt,1.778058,3.433815,1.879095,0.586836,2.630216,-0.185258,2.249052,95.451606,321.865860,-400.614630,6.250000,4.922717,14.870578,200.181148,1.448251,6.099553


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df.select_dtypes(include=['float64', 'int64'])
#X = X.drop(['min_log_velocity','max_log_velocity','max_acceleration', 'min_acceleration', 'max_curvature'], axis=1)


# Replace NaN values with the mean of each column
X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)

# Confirm there are no NaN values left
print("Remaining NaNs:", X_filled.isna().sum().sum())  # Should output 0


Remaining NaNs: 0


In [59]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)
y = df['track_type']
X_train, X_test, y_train, y_test = train_test_split(X_filled, y, test_size=0.1, random_state=42)


In [60]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # StandardScaler for preprocessing
    ('mlp', MLPClassifier(hidden_layer_sizes=(50, 30), activation='tanh', 
                          solver='adam', alpha=0.001, learning_rate_init=0.1, 
                          max_iter=20000, early_stopping=False))
])




# Train the model on the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 62.20%


In [61]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Define a pipeline without grid search
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('pca', PCA(n_components=11)),  # Use 15 principal components to reduce dimensionality
    ('classifier', VotingClassifier(  # Ensemble of three classifiers
        estimators=[
            ('mlp', MLPClassifier(max_iter=1000, hidden_layer_sizes=(50, 30), alpha=0.001, learning_rate_init=0.01)),
            ('rf', RandomForestClassifier(n_estimators=100)),
            ('gb', GradientBoostingClassifier(learning_rate=0.1, n_estimators=100))
        ],
        voting='soft'  # Soft voting to average probabilities from all classifiers
    ))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Print the accuracy score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 68.29%


In [62]:
from sklearn.model_selection import GridSearchCV

# Simplified parameter grid
param_grid = {
    'pca__n_components': [10, 11],  # Fewer PCA components to test
    'classifier__mlp__hidden_layer_sizes': [(50, 30), (60, 40)],  # Fewer hidden layer sizes for MLP
    'classifier__rf__n_estimators': [100, 150],  # Only two options for RandomForest trees
    'classifier__gb__n_estimators': [100, 150]  # Fewer boosting stages for GradientBoosting
}

# Initialize GridSearchCV with fewer options and a 3-fold CV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation to reduce time
    n_jobs=-1,  # Use all available cores
    scoring='accuracy'  # Use accuracy as the metric
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score: {:.2f}%".format(grid_search.best_score_ * 100))

# Predict on the test set with the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print the accuracy score of the best model
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with Best Model: {accuracy * 100:.2f}%")


Best Parameters: {'classifier__gb__n_estimators': 100, 'classifier__mlp__hidden_layer_sizes': (50, 30), 'classifier__rf__n_estimators': 150, 'pca__n_components': 11}
Best Cross-Validation Score: 59.29%
Test Accuracy with Best Model: 70.73%


array([[ 9,  1,  7,  0,  0,  0],
       [ 0, 12,  0,  3,  0,  1],
       [ 7,  0, 19,  0,  0,  0],
       [ 0,  4,  0,  2,  0,  2],
       [ 0,  0,  0,  0,  1,  1],
       [ 0,  3,  0,  0,  0, 10]])